## This notebook is to create the Dataframe with Canada's Posta codes, Borough and Neigborhood. The Dataframe will be reformatted as suggested in the section 1 of this assignment.

#### Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

#### Using Requests package get the required wikipedia page in the xml format. Then using lxml and BeautifulSoup packages get the list of Postal Codes, Borough and Neighborhood values and create a python list.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
src_xml = BeautifulSoup(source, 'lxml')
table = src_xml.find('table', class_="wikitable sortable")
tbody1 = table.findAll('td')
list = []
for vals in tbody1:
    list.append(vals.text)

#### Define the column names of the Dataframe

In [3]:
cols=['PostalCode', 'Borough', 'Neighborhood']
canada_df = pd.DataFrame(columns=cols)

#### Since the Posta codes, Borough and Neighborhood are in the same list, separate them in to different python lists and create the Dataframe

In [4]:
x = 0
cols1=[]
cols2=[]
cols3=[]
for s in range(0,280):
    cols1.append(list[x])
    x+=1
    cols2.append(list[x])
    x+=1
    cols3.append(list[x].replace('\n', ''))
    x+=1

canada_df['PostalCode']=cols1
canada_df['Borough']=cols2
canada_df['Neighborhood']=cols3

#### Remove the rows where Borough has the value 'Not assigned'

In [5]:
canada_df=canada_df.set_index('Borough')
canada_df=canada_df.drop('Not assigned', axis=0)
canada_df=canada_df.reset_index()
col_lst=['PostalCode', 'Borough', 'Neighborhood']
canada_df=canada_df[col_lst]

In [6]:
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### There can be multiple Neighborhood for the same Postal code. So Group them by the Postal code/Borough and combine the rows such that there is only 1 row per Postal code with all the Neighborhood sparated by comma

In [7]:
canada_grp=canada_df.groupby(['PostalCode','Borough'], as_index=False, sort=True).agg(', '.join)

In [8]:
canada_grp.tail(20)

,PostalCode,Borough,Neighborhood
82,M6P,West Toronto,"High Park, The Junction South"
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Not assigned
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


#### There are many Neighoborhoods with no values (with 'Not assigned'). So assign the corresponding Borough values to the Neighborhood

In [9]:
for inx in canada_grp.index:
    if canada_grp['Neighborhood'][inx] == 'Not assigned':
        canada_grp['Neighborhood'][inx] = canada_grp['Borough'][inx]

canada_grp.tail(20)

,PostalCode,Borough,Neighborhood
82,M6P,West Toronto,"High Park, The Junction South"
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [10]:
canada_grp.shape

(102, 3)

#### Import the CSV file with the Latitude and Longitude for the Post codes. And then merge the new Dataframe with the earlier dataframe so that we have the Postal codes, Borough, Neighborhood and corresponding Latitute/Longitude values in the single Dataframe

In [14]:
latlong_df = pd.read_csv(r'''C:\Users\sharath\Desktop\DataS_cert\Geospatial_Coordinates.csv''')
latlong_df.columns = ['PostalCode','Latitude','Longitude']
result_df = pd.merge(canada_grp,latlong_df,on='PostalCode',how='left')
result_df.columns = ['PostalCode','Borough', 'Neighborhood','Latitude','Longitude']
result_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
